In [ ]:
import os

In [ ]:
os.chdir("../../")

In [ ]:
import re

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
# from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from textwrap import dedent

from src.initialization import credential_init
from src.io.path_definition import get_project_dir


def build_standard_chat_prompt_template(kwargs):

    messages = []
 
    if 'system' in kwargs:
        content = kwargs.get('system')
        prompt = PromptTemplate(**content)
        message = SystemMessagePromptTemplate(prompt=prompt)
        messages.append(message)  

    if 'human' in kwargs:
        content = kwargs.get('human')
        prompt = PromptTemplate(**content)
        message = HumanMessagePromptTemplate(prompt=prompt)
        messages.append(message)
        
    chat_prompt = ChatPromptTemplate.from_messages(messages)
    
    return chat_prompt



credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", temperature=0)

# Read file
filename = os.path.join("tutorial", "week_1", "唐詩三百首.txt")
with open(filename, "r", encoding="utf-8") as f:
    text = f.read()

poems = []

# Split by blank lines
blocks = [b.strip() for b in text.strip().split("\n\n") if b.strip()]

for block in blocks:
    entry = {}
    for line in block.split("\n"):
        if line.startswith("詩名:"):
            entry["詩名"] = line.replace("詩名:", "").strip()
        elif line.startswith("作者:"):
            entry["作者"] = line.replace("作者:", "").strip()
        elif line.startswith("詩體:"):
            entry["詩體"] = line.replace("詩體:", "").strip()
        elif line.startswith("詩文:"):
            entry["詩文"] = line.replace("詩文:", "").strip()
    if len(entry) != 0:
        poems.append(entry)

# 語意檢索系統

> 🎯 **本章學完你將能學會什麼：**
> - 理解「語意檢索」的核心概念與傳統單詞比對的限制  
> - 了解 **Word2Vec**、**GloVe** 的詞向量原理與語義空間表示方式  
> - 認識 **上下文詞向量（Contextual Embeddings）** 與 **Transformer 架構**  
> - 比較 Encoder-only（BERT）、Decoder-only（GPT）、Encoder-Decoder（T5）的差異  
> - 掌握如何在 Python 中使用 **Gensim**、**LangChain** 與 **FAISS** 進行語意檢索  
> - 能將文字資料轉換為向量嵌入，並建立可檢索的語意資料庫  
> - 應用語意搜尋於詩文檢索、文本對比與自動化問答等任務  



使用單詞比對搜尋的問題:

猛男 != 兄貴

我們知道這在意思上非常的相近，但是單詞比對不一樣就是不一樣，差一點點都不行

## 解決方法:

Word2Vec

- Word2Vec 可透過 Skip-gram 或 CBOW 架構，將單詞映射到實數向量空間。
- 在這個空間中，單詞的分佈式表示能捕捉語義與上下文關係。

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(filename= "tutorial/week_2/Word2Vec.png")

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api

# Download and load the model (first-run will download ~128 MB)
model = api.load('glove-wiki-gigaword-100')

# Usage examples:
# king + woman - man
print(model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1))
# → [('queen', similarity_score)]

# print(model['computer'][:5])

In [ ]:
print(model['queen'])

In [ ]:
# Usage examples:
print(model.most_similar(positive=['warrior', 'woman'], negative=['man'], topn=1))

In [ ]:
# Usage examples:
print(model.most_similar(positive=['priest', 'woman'], negative=['man'], topn=1))

In [ ]:
# 這個模型1.5G大小

# from gensim.models import KeyedVectors

# model = KeyedVectors.load_word2vec_format(
#     'GoogleNews-vectors-negative300.bin', binary=True
# )

In [ ]:
# Cosine Similarity 高中數學沒忘光吧
word1 = "king"
word2 = "queen"

similarity_score = model.similarity(word1, word2)
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity_score:.4f}")

In [ ]:
# Most similar words

model.most_similar('king', topn=5)

Word2Vec 雖然能捕捉詞彙的語義相似性，但它有一個限制：無法處理一字多義(polysemy)的情況。因為 Word2Vec 是全域向量(global vector)模型，每個單詞只有一個固定的向量表示，與上下文無關。例如：
- 菜市場的「蘋果」和 San Jose 的「Apple」的向量是一樣的
- 河岸的「bank」和銀行的「bank」向量也是一樣的

解決方法是使用上下文詞向量（Contextual Embeddings），如 BERT 或 ELMo，它們能根據上下文生成不同的詞向量。

### Transformers

In [ ]:
from IPython.core.display import HTML 
Image(url= "https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)#/media/File:Transformer,_full_architecture.png")

Transformer的Attention layer可以讓單詞去注意他該去"看"句子的哪個地方，來達到捕捉上下文涵意

#### Attention 的作用

Transformer 的 Attention 層可以讓每個單詞在編碼時「注意」句子中其他單詞的重要性，從而捕捉上下文語意。

#### Transformer 架構

- Encoder-Decoder (T5)：Encoder 負責理解輸入句子，Decoder 負責生成文字。

- Encoder-only（如 BERT）：只做理解，產生 Contextual Embeddings。

- Decoder-only（如 GPT）：只做生成，但也能捕捉上下文語意。

Contextual Embeddings

每個單詞的向量表示(vector presentation)會根據上下文不同而改變。

這些表示可以由 Encoder 或 Decoder 生成，取決於模型架構。

Embedding 的獲得方式

通常由 Transformer 在訓練過程中學得（例如 Masked Language Modeling、Causal LM 等）。

也可以用特殊任務或對比學習方法訓練句子/單詞嵌入，但這不是唯一方法。

### Contrastive Learning


1. 目標

在嵌入（Embedding）學習中，我們希望把輸入資料（如文字、圖片、音訊）映射到一個向量空間中，使得：

相似的樣本在向量空間中距離接近

不相似的樣本在向量空間中距離遠

對比學習正是用這個原理來訓練模型生成高品質的 Embedding。

2. 做法

選取正負樣本對

正樣本對（positive pair）：同一個語義或內容的兩個樣本。

文字：同義句、翻譯句

圖片：同一張圖片經過不同增強（裁切、旋轉、顏色變化）

負樣本對（negative pair）：語義不同或來源不同的樣本

映射到向量空間

用模型（如 Transformer、CNN）將每個樣本映射成向量表示（Embedding）

計算正負樣本對的相似度（通常用餘弦相似度）

訓練目標

拉近正樣本對的距離

推遠負樣本對的距離

損失函數常用 Contrastive Loss 或 InfoNCE Loss

3. 效果

訓練完成後，Embedding 空間會自動形成 語義結構：

相似意思的文字或圖片會聚在一起

不相似的樣本會遠離

這種 Embedding 可以直接用於：

檢索（Retrieval）：找最相似的句子或圖片

分類（Classification）：用 Embedding 做下游任務

聚類（Clustering）：將相似樣本聚在一起

4. 實際例子

文字嵌入：Sentence-BERT 用對比學習將語義相近的句子映射到相近向量，方便語義搜索或問答系統。

圖像嵌入：SimCLR 用對比學習生成圖片特徵向量，無需標籤也能捕捉圖像語義。

多模態嵌入：CLIP 將圖片和對應描述映射到同一向量空間，方便圖像檢索或生成。

理論講完了，上手實操。理論建議看一下，也許面試時會問。

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

# A list of embedding models you can choose 
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

### 1. 創建嵌入:

In [ ]:
# embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### 2. 載入向量資料庫:

In [ ]:
import pandas as pd
from langchain_core.documents import Document

df_poem = pd.DataFrame(poems)

documents = []

for _, row in df_poem.iterrows():
    document = Document(page_content=row['詩文'],
                        metadata={"詩名": row["詩名"],
                                  "作者": row["作者"],
                                  "詩體": row["詩體"]})
    documents.append(document)

filename = os.path.join(get_project_dir(), "tutorial", "week_2", "poem_faiss_index")

In [ ]:
# 挑選一個有包含中文的Embedding 模型
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 我的這台電腦要跑一個小時左右，所以留給你們，反正不用錢
# 我直接使用已經轉換完成的 vectorstore
vectorstore = FAISS.from_documents(documents, embedding=embeddings)

# it seems that you cannot use Chinese character in the filename
vectorstore.save_local(filename)

In [ ]:
# vectorstore = FAISS.load_local(
#     filename, embeddings, allow_dangerous_deserialization=True
# )

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

retriever.invoke("夕陽無限好")

## 🧠 理解 FAISS 中的 IVF-PQ

> 🎯 **本章學完你將能學會什麼：**
> - 理解 **FAISS（Facebook AI Similarity Search）** 的基本架構與用途  
> - 分辨不同索引類型：**IndexFlatL2、IVF、PQ、IVF+PQ**  
> - 了解倒排索引（IVF）與產品量化（PQ）的運作機制  
> - 掌握 **IVF+PQ** 結合後的優勢：高速檢索與記憶體壓縮  
> - 能在程式中建立、訓練與儲存 FAISS 索引  
> - 學會如何使用 LangChain 的 **FAISS VectorStore** 封裝檢索流程  
> - 理解三種檢索策略（similarity、score threshold、MMR）與實際應用差異  
> - 具備分析向量空間結構與相似度分佈的能力  

本筆記說明 **FAISS**（Facebook AI Similarity Search）的索引與搜尋策略，特別聚焦在 **IVF**、**PQ** 及其組合 **IVF+PQ**。

---

### 🧩 IndexFlatL2

**機制：**  
最簡單的暴力搜尋索引 — 查詢時，會計算查詢向量與 **資料庫中所有向量** 的 **L2 距離**，然後排序結果。

**優點：**
- 提供 **精確最近鄰**。
- 不需要訓練（可即插即用）。

**缺點：**
- 對於大規模資料集（百萬或數十億向量）非常 **慢**，且 **耗記憶體**。

---

### ⚙️ IVF（Inverted File Index 倒排索引）

**概念：**  
將向量空間分成 `nlist` 個 cluster（通常使用 **k-means** 聚類）。  
每個 cluster 有一個 **倒排列表**（inverted list），儲存分配到該 cluster 的向量。

**查詢流程：**
1. 使用 **coarse quantizer**（通常是 `IndexFlatL2`）找出與查詢向量最接近的 `nprobe` 個 cluster。  
2. 僅在這些 cluster 中進行細部搜尋，而非整個資料庫。

**優點：**
- 透過忽略大部分資料庫，大幅 **加快搜尋速度**。
- 適合 **大規模資料集**。

**缺點：**
- 精準度略下降（取決於 `nlist` 與 `nprobe` 的設定）。
- 需要訓練（k-means）。

---

### 📦 PQ（Product Quantization 產品量化）

**概念：**  
將向量壓縮成緊湊表示，以便更快地進行近似距離計算。

**步驟：**
1. 將向量（維度 `d`）分成 `m` 個 **子向量**，每個長度為 `d/m`。
2. 為每組子向量訓練一個 **codebook**（centroids 集合），通常大小為 `2^bits`。
3. 將每個子向量替換為最接近 centroid 的 **索引值**。

如此一來，一個浮點向量就可以轉換為一串小整數編碼，大幅減少記憶體使用量。

**查詢時：**
- 不直接計算向量距離，而是使用查詢向量各子向量與 codebook 的 **查表距離**。
- 將每個子向量的距離加總作為近似距離。

**優點：**
- **大幅壓縮記憶體**（可縮小 10–100 倍）。
- **加速距離計算**。

**缺點：**
- 會產生量化誤差，降低精確度。

---

### 🔗 IVF + PQ（混合索引）

**概念：**
- **IVF**：縮小搜尋範圍（只看部分 cluster）。
- **PQ**：壓縮 cluster 內向量，加快運算。

**結果：**
- **搜尋速度快**、**記憶體效率高**。
- 精確度略有下降，但通常可接受。

**典型應用：**  
大規模 **語意搜尋**、**embedding 檢索** 或 **影像相似度搜尋**，需要高效率搜尋大量向量。

---

### ✅ 總結

| 索引類型       | 描述                                   | 精準度 | 速度 | 記憶體 | 是否需要訓練 |
|----------------|----------------------------------------|---------|------|--------|----------------|
| **IndexFlatL2** | 精確暴力搜尋                           | ✅ 高    | ❌ 慢 | ❌ 大   | 否             |
| **IVF**         | 基於 cluster 的倒排搜尋                 | ⚖️ 中   | ✅ 快 | ✅ 較小 | 是             |
| **PQ**          | 向量量化與壓縮                          | ⚖️ 中   | ✅ 快 | ✅ 最小 | 是             |
| **IVF + PQ**    | Cluster + Quantization（混合）          | ⚖️ 中   | ✅✅ 非常快 | ✅✅ 緊湊 | 是             |

---

> 🧭 **簡而言之：**  
> - `IndexFlatL2` → 精確但慢。  
> - `IVF` → 篩選搜尋範圍。  
> - `PQ` → 壓縮向量與加速計算。  
> - `IVF + PQ` → 結合兩者，實現**快速、節省記憶體**的近似最近鄰搜尋。

---

In [ ]:
import faiss

# The raw FAISS index
faiss_index = vectorstore.index  

# Extract all vectors from the FAISS index
# (returns numpy array of shape [n_vectors, dim])
vectors = faiss_index.reconstruct_n(0, faiss_index.ntotal)

dimension = vectors.shape[1]

# -----------------------------
# 3. 建立 IVF + PQ FAISS 索引
# -----------------------------
nlist = 1  # 聚類中心數量
m = 1      # PQ 分段數
bits = 8
quantizer = faiss.IndexFlatL2(dimension)  # 基礎量化器

# IndexIVFPQ：IVF + PQ 索引
index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, bits)
index.train(vectors)
index.add(vectors)

In [ ]:
from langchain.docstore import InMemoryDocstore

# -----------------------------
# 4. 封裝到 LangChain FAISS
# -----------------------------
# 這裡我們直接把 faiss 索引和文本對應起來
faiss_db = FAISS(
    embedding_function=embeddings.embed_query,  # 嵌入函數
    index=index,
    docstore=InMemoryDocstore(
        {i: doc for i, doc in enumerate(documents)}
    ),
    index_to_docstore_id={i: i for i in range(len(documents))}
)

In [ ]:
retriever_ann = faiss_db.as_retriever(search_kwargs={'k': 5})
retriever_ann.invoke("夕陽無限好")

## Runtime Configuration

複習上星期學到的東西: 透過Runtime Configuration動態的調整搜索器的參數

In [ ]:
from langchain_core.runnables import ConfigurableField

query = "夕陽無限好"

retriever = vectorstore.as_retriever(search_type="similarity").configurable_fields( \
                                        search_kwargs=ConfigurableField(
                                                id="hello_search",
                                            )
                                        )

In [ ]:
retriever.invoke(query, config={"configurable": {"hello_search": {"k": 7}}})

In [ ]:
retriever.invoke(query, config={"configurable": {"hello_search": {"k": 3}}})

## Three search types:

### 1. similarity (default)

- 這種搜索類型找到與你的查詢最相似的文檔。它會看你使用詞語的意思，並匹配具有相似意思的文檔。可以把它想像成找到與你感興趣的主題密切相關的文章或文檔。

### 2. similarity_score_threshold (相似性分數閾值):

- 這種搜索類型設置一個最小相似性分數，只有達到這個分數的文檔才會被認為是相關的。只有那些在意思上與你的查詢非常接近的文檔才會被包含進來。它確保結果高度相關，並過濾掉不太相關的信息。

In [ ]:
"""
cosine similarity

https://api.python.langchain.com/en/latest/_modules/langchain_core/vectorstores.html

elif search_type == "similarity_score_threshold":
    docs_and_similarities = self.similarity_search_with_relevance_scores(
        query, **kwargs
    )
    return [doc for doc, _ in docs_and_similarities]

in subclass.
Return docs and relevance scores in the range [0, 1].

0 is dissimilar, 1 is most similar.
"""

query = "傍晚心情鬱悶，登高望景。眼前的夕陽極為壯麗，但它即將沉沒，這就像人生或理想，美好卻轉瞬即逝"

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2, "k": 5}
)

retriever.invoke(query)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "k": 5}
)

retriever.invoke(query)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k": 5}
)

retriever.invoke(query)

### 如何取得文檔的分數?

In [ ]:
query = "傍晚心情鬱悶，登高望景。眼前的夕陽極為壯麗，但它即將沉沒，這就像人生或理想，美好卻轉瞬即逝"

vectorstore.similarity_search_with_score(query)

In [ ]:
query = "傍晚心情鬱悶，登高望景。眼前的夕陽極為壯麗，但它即將沉沒，這就像人生或理想，美好卻轉瞬即逝"

vectorstore._similarity_search_with_relevance_scores(query)

我們可以看到透過_similarity_search_with_relevance_scores 我們可以取得 cosine similarity 的分數

### 試試看翻譯跟原文的相似度有多少

In [ ]:
query = "傍晚時分，心中感到鬱悶不快，我便驅車登上古老的高原。極目遠望，晚霞映照的夕陽格外壯麗動人，然而這份美景卻接近黃昏，轉瞬將要消逝。"

vectorstore._similarity_search_with_relevance_scores(query)

In [ ]:
query = "人生中最美好的事物，往往正如夕陽般壯麗，卻難免走向衰落與消亡。"

vectorstore._similarity_search_with_relevance_scores(query)

In [ ]:
query = dedent("""
傍晚時分，心裡難以言說的惆悵悄然浮起，我便乘車登上那片古老的高原。天地遼闊，霞光萬丈，夕陽的光彩無比絢麗，彷彿要將最後的熱情全部燃盡。可惜它再怎麼燦爛，也只是黃昏的前奏，美麗正因將逝而更添傷感。
""")

vectorstore._similarity_search_with_relevance_scores(query)

In [ ]:
# 這些分數在幾何上看起來如何?

import math

print(math.acos(0.5351006626402812)/math.pi * 180)

print(math.acos(0.6803903535716722)/math.pi * 180)

有沒人有想要挑戰能不能超過0.68?

** 自動化評分系統 **

國文考試喜歡問作者在想啥，但是不同的老師評分標準可能不一樣。
那乾脆用一個Embedding系統來計分，保證公平性。

### 3. MMR, Maximum Marginal Relevance (MMR, 最大邊際相關性):

- 這種方法在找到與你的查詢相似的文檔的同時，也確保結果是多樣的。這就像是在一個主題上尋求多種意見，避免得到過多相同的東西。它有助於避免搜索結果的冗餘。

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/v2/resize:fit:720/format:webp/1*c0c19i2tPSWZaHwQ7cVMrg.png")

In [ ]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 50, 'lambda_mult': 0.1})

In [ ]:
query = "傍晚時分，心中感到鬱悶不快，我便驅車登上古老的高原。極目遠望，晚霞映照的夕陽格外壯麗動人，然而這份美景卻接近黃昏，轉瞬將要消逝。"

retriever.invoke(query)

### 如何使用Metadata?

In [ ]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 50, 'lambda_mult': 0.1,
                                                                       "filter": {'詩體': "五言絕句"}})

query = "傍晚時分，心中感到鬱悶不快，我便驅車登上古老的高原。極目遠望，晚霞映照的夕陽格外壯麗動人，然而這份美景卻接近黃昏，轉瞬將要消逝。"

retriever.invoke(query)

更多古典詩在這裡，你有辦法把他們抓下來並且整理成Structured Data嗎?

https://github.com/rainrambler/QTS/blob/main/qts_zht.txt

## 複數條件過濾

In [ ]:
df_poem[df_poem["作者"]=='李商隱']

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 3,
                                                                              'filter': {'詩體': "五言律詩",
                                                                                         '作者': '李商隱'}})

T = retriever.invoke(query)

print(T)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 3, 'fetch_k': 100,
                                                                              'filter': {'詩體': "五言律詩",
                                                                                         '作者': '李商隱'}})

T = retriever.invoke(query)

print(T)

這是因為FAISS 使用 Post-filter: 先使用相似度排序前fetch_k，然後再用filter過濾。

可以試試看其他的檢索器，像是QRANT (沒正式在工作上使用過)

In [ ]:
!pip install -qU langchain-qdrant

In [ ]:
# 直接從FAISS vectorstore將vectors抽取出來
faiss_index = vectorstore.index  

# Extract all vectors from the FAISS index
# (returns numpy array of shape [n_vectors, dim])
vectors = faiss_index.reconstruct_n(0, faiss_index.ntotal)

dimension = vectors.shape[1]

In [ ]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams


client = QdrantClient(path="/tmp/langchain_qdrant")

collection_name = "demo_collection"

# Qdrant server初始化
try:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=Distance.COSINE),
    )
except ValueError:
    client.delete_collection(collection_name=collection_name)
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=Distance.COSINE),
    )
"""
Method create_collection is defining a new vector collection inside Qdrant (kind of like creating a table in SQL).
- Creates a named collection in Qdrant where your vectors + metadata will live.
- Each collection has its own configuration: vector dimensionality, distance metric, storage options, etc.

collection_name: just a string identifier (like a table name).
- You’ll later refer to it in your QdrantVectorStore and queries.
- In your example: "demo_collection" is the container where all vectors will be stored.

The dimension of your embedding vectors (must match your embedding model’s output size).

Remove collection:
client.delete_collection(collection_name="demo_collection")
"""

# 將數據上載到server裡
client.upsert(
    collection_name="demo_collection",
    points=[models.PointStruct(id=idx + 1,
                               vector=vector,
                               payload={"page_content": document.page_content,
                                        "metadata": document.metadata}) for idx, (document, vector) in enumerate(zip(documents, vectors))]
)

# 建立 vectorstore
vectorstore_QVS = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

### 挑戰: 你可以使用method from_documents 來創立一個vectorstore嗎?

In [ ]:
retriever_QVS = vectorstore_QVS.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retriever_QVS.invoke("夕陽無限好")

### Qdrant透過filter過濾內容

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

filter_ = Filter(
    must=[
        FieldCondition(key="metadata.作者", match=MatchValue(value="李商隱")),
        FieldCondition(key="metadata.詩體", match=MatchValue(value="五言律詩")),
    ]
)

retriever_QVS = vectorstore_QVS.as_retriever(search_type="similarity", search_kwargs={"k": 5, 
                                                                                      'filter': filter_})
retriever_QVS.invoke("夕陽無限好")

### Question 

假設你寫了一個穿越小說，角色穿越到了古代，為了凸顯主角的文學素養，你希望他可以吟詩作對。
如何寫一個小App，根據需求，來幫助你完成這件事情。

(15分鐘)

In [ ]:
from typing import Literal

In [ ]:
df_poem['詩體'].unique().tolist()

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

from langchain_core.output_parsers import PydanticOutputParser

class requirements(BaseModel):

    question: str = Field(description="")
    answer: Literal['五言古詩', '七言古詩', '七言律詩', '五言絕句', '樂府', '七言絕句', '五言律詩'] = Field(description="")

    

In [ ]:
output_parser = PydanticOutputParser(pydantic_object=requirements)

format_instructions = output_parser.get_format_instructions()


system_template = dedent("""
                  """)

human_template = dedent("""
                 format instruction: {format_instructions}
                 """)

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": [],
                    "partial_variables": {"format_instructions": 
                                          format_instructions}}}

requirement_prompt_template = build_standard_chat_prompt_template(input_)

# LangChain Expression Language (LCEL)

> 🎯 **本章學完你將能學會什麼：**
> - 理解 **LangChain Expression Language (LCEL)** 的設計理念與語法特性  
> - 掌握 LCEL 如何以「鏈式運算子 `|`」實現資料流的模組化組合  
> - 熟悉 **Runnable** 的核心概念，了解各類 Runnable（如 `RunnableLambda`、`RunnableParallel`、`RunnablePassthrough`）的用途  
> - 學會構建多階段 Pipeline：從 Prompt → LLM → Parser 的自動化串接  
> - 能運用 LCEL 進行 **生成 → 評估 → 改進（Reflection Agent）** 的循環架構設計  
> - 理解 **Runtime Configuration** 的動態參數傳遞方法，實現可重組與條件化檢索  
> - 掌握 LCEL 在 **語意檢索（Semantic Retriever）** 中的整合實務  
> - 能夠將同步與異步流（Synchronization / Asynchronization）轉換，以提升高併發執行效率  
> - 學會使用 LCEL 建立 **平行化工作流（RunnableParallel）** 與 **自定義任務鏈**  
> - 了解如何利用 LLM 實現 **程式生成（Code Generation）** 與自動執行（`exec()`）  
> - 綜合應用 LCEL 的思維，設計可擴展、可維護的 AI 工作流架構  


**LangChain Expression Language (LCEL)** 是一種強大且靈活的語法，用於建構高效的資料處理流水線（Pipeline）。它允許開發者以鏈式（Chain）方式將多個操作（如檢索、生成、處理）無縫串聯，將前一個節點的輸出作為下一個節點的輸入，形成流暢的資料流。LCEL 的設計強調模組化、可讀性和可維護性，特別適用於結合大型語言模型（LLM）與外部工具（如向量資料庫、API）來解決複雜任務。

## 為什麼使用 LCEL？
- **高效能與可擴展性**：透過異步處理（Asynchronous Flow）與並行執行（Parallelization），LCEL 能在高併發場景下顯著提升效率。
- **簡潔的語法**：使用 `|` 運算子串聯操作，代碼簡潔且易於理解，類似於 Unix 的管道（Pipe）概念。
- **整合性強**：LCEL 能輕鬆整合 LangChain 的各種組件（如 Retriever、PromptTemplate、OutputParser）與外部工具（如 FAISS、Qdrant）。

## 核心概念
1. **Runnable 物件**：LCEL 的基本單位，代表一個可執行的操作（如檢索、模型推理、資料轉換）。常見的 Runnable 包括 `RunnablePassthrough`（保留輸入）、`RunnableLambda`（自訂函數）等。
2. **流水線操作**：使用 `|` 將多個 Runnable 串聯，前一階段的輸出自動傳遞到下一階段。例如：`prompt | model | parser`。
4. **並行與異步**：支援 `RunnableParallel` 同時執行多個獨立任務，以及 `ainvoke` 方法進行異步處理，適用於高吞吐量應用。

In [ ]:
## Official diagram flow

Image(filename= "tutorial/LLM+Langchain/Week-2/lcel pipeline.png")

## Base of Reflection Agent

- https://huggingface.co/blog/Kseniase/reflection
- https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/reflection/reflection.ipynb

generate → critique → improve

In [ ]:
from langchain_core.output_parsers import StrOutputParser

query = dedent("""
俗話說：「龍生九子，各有不同。」在廣闊浩瀚的海洋之中，就有一頭孤獨的鯨魚——五十二赫茲鯨魚。牠聲音的頻率天生便比同伴還要高，這項特別之處，也導致了牠與同伴產生了無法溝通的鴻溝。看見這則故事的我，不禁思考，在如此多元的人間，是否也有像五十二赫茲鯨魚一般，天生便與眾不同？

回首童年，我印象最為深刻的一刻，是初識字時，與文字互相理解的那一瞬、是當我第一次讀完一個句子時，它將自身的意義傳入我腦中的那一瞬。自此，我便對文字、語言抱有特殊的感情，也十分享受閱讀與朗誦。那種將自身與文字經由一點一滴積累而連接起來的感情，使我心靈感到十分富足。

而當我步入校園接觸同儕時，驀然驚覺我與別人的閱讀速度十分不同。每當我已讀完一篇文章，但同學可能只完成了一半甚至三分之二。同時，我在生字讀音方面也異常的執著，因此被同學抱怨有「文字潔癖」。面對同儕抱怨的我，也只好強忍對耳邊時而出現字錯讀音的不適，開始刻意忽略心裡對它的執念，只為想要與別人一樣，想要和朋友互相理解。

直到多年前，因緣際會之下認識了「五十二赫茲」這獨特的存在。牠的身影在我心中烙下一道深刻的痕跡。因為牠，我開始接受自己與他人的不同；也因為牠，我明白了，我對文字的執著，並不是一種負面的特質，而是上天賜予我的禮物，我開始在寫作上揮灑自如。這讓我知道，不要在一開始便用否定的眼光看待自己的特質。也許這特別之處，會使我們與五十二赫茲鯨魚一般孤獨，會使我們遭受他人的不理解與排斥，但也會讓我們與眾不同。

關於此，我想說的是，勇敢地綻放自己的特別，也讓自己成為自己和世人眼中，最閃耀的五十二赫茲鯨魚。
""")

## Teacher LLM
system_template = dedent("""
你是一個教學與寫作經驗豐富的台灣大學中文系教授，你要來負責給予作文評分與回饋。
""")

human_template = dedent("""
Title: {title}

Article:
{article}
""")

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", temperature=0)

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["title", "article"],
                    }}

chat_prompt_template = build_standard_chat_prompt_template(input_)

chat_prompt = chat_prompt_template.invoke({"article": query,
                                           "title": "關於五十二赫茲，我想說的是…"})

feedback = model.invoke(chat_prompt)

print(feedback.content)

In [ ]:
# 將所有邏輯用 pipeline 符號 |

feedback_pipeline = chat_prompt_template|model|StrOutputParser()

feedback = feedback_pipeline.invoke({"article": query,
                                     "title": "關於五十二赫茲，我想說的是…"})

print(feedback)

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Output(BaseModel):
    name: str = Field(description="The revised article in traditional Chinese (繁體中文), please do not include the title.")

output_parser = PydanticOutputParser(pydantic_object=Output)
format_instructions = output_parser.get_format_instructions()

## Generate
system_template = dedent("""
你是一個在準備考試的高中生，你將根據反饋強化作文的內容。
""")

human_template = dedent("""
Title: {title}

Old Article:
{article}

Feedback:
{feedback}

Output instruction: {format_instructions}

Revised Article:
""")

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", temperature=0)

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["title", "article", "feedback"],
                    "partial_variables": {'format_instructions': format_instructions}
                    }}

chat_prompt_template = build_standard_chat_prompt_template(input_)

revision_pipeline = chat_prompt_template|model|output_parser

revised_result = revision_pipeline.invoke({"article": query,
                                           "title": "關於五十二赫茲，我想說的是…",
                                           "feedback": feedback})

print(revised_result)

In [ ]:
print(revised_result.name)

In [ ]:
feedback = feedback_pipeline.invoke({"article": revised_result.name,
                                     "title": "關於五十二赫茲，我想說的是…"})

print(feedback)

### 如何結合 feedback_pipeline 和 revision_pipeline?

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassThrough: 酒肉穿腸過 佛祖心中留
# 技術一點的說法: 它是一個 no-op（無操作）的 runnable，常用來保留數據流的連貫性，或作為調試／替換的 placeholder。

whole_pipeline = RunnablePassthrough.assign(feedback=feedback_pipeline)|revision_pipeline

revised_version = whole_pipeline.invoke({"article": query,
                                         "title": "關於五十二赫茲，我想說的是…"})

print(revised_version.name)

assign 就像是 在 Pipeline 中加入一個「中途站」或「資訊亭」，在主數據流（例如 article, title）繼續向前時，它同時執行一個子流程（例如 feedback_pipeline），並將子流程的結果（feedback）以新的 Key 注入到主數據流中，以便後續步驟使用。

這是Generate - Reflection的雛型。可以隨需求將它的架構弄得更複雜。

## 延續上周

上週我們使用BM25做Retriever
這次我們把BM25替換成Semantic Retriever

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Output(BaseModel):
    name: str = Field(description="The generated poem in traditional Chinese (繁體中文)")

output_parser = PydanticOutputParser(pydantic_object=Output)
format_instructions = output_parser.get_format_instructions()


query = dedent("""
蒹葭蒼蒼、白露為霜。
所謂伊人、在水一方。
遡洄從之、道阻且長。
遡遊從之、宛在水中央。
""")

system_template = dedent("""
You are a helpful AI assistant with expertise in classical Chinese literature.
You understand all the nuance and history background of all the content.
""")

human_template = dedent("""
Create a {poetic_form} which shares the same semantic of {query}

Examples of {poetic_form}:

{context}

Output instruction: {format_instructions}
""")


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query", "context", "poetic_form"],
                    "partial_variables": {'format_instructions': format_instructions}}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

### Runtime Configuration (運行時配置)

In [ ]:
from langchain_core.runnables import ConfigurableField

poetic_form = "五言絕句"

retriever_QVS = vectorstore_QVS.as_retriever(search_type="similarity")
retriever_QVS = retriever_QVS.configurable_fields(search_kwargs=ConfigurableField(id="qdrant_search_kwargs"))

filter_ = Filter(
    must=[
        FieldCondition(key="metadata.詩體", match=MatchValue(value=poetic_form)),
    ]
)

semantic_documents = retriever_QVS.invoke(query, config={"configurable":
                                                          {"qdrant_search_kwargs":{
                                                                "k": 5,
                                                                "filter": filter_}}})

semantic_documents

In [ ]:
context = "\n\n".join([c.page_content for c in semantic_documents])

print(context)

In [ ]:
chat_prompt = chat_prompt_template.invoke({"query": query,
                                           "poetic_form": "五言絕句",
                                           "context": context})
print(chat_prompt)

In [ ]:
model_poem = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                        model_name="gpt-4o", temperature=0)

output = model_poem.invoke(chat_prompt)
output_parser.parse(output.content)

上述的操作中有兩個挑戰:
1. semantic_documents = retriever_QVS.invoke(query, config={"configurable":
                                                          {"qdrant_search_kwargs":{
                                                                "k": 5,
                                                                "filter": filter_}}})




   沒有辦法直接pass config (至少我目前還不知道)

3. context = "\n".join([c.page_content for c in semantic_documents])

   這一步不是一個Runnable


解法: 套殼

In [ ]:
from operator import itemgetter

from langchain_core.runnables import chain, RunnableLambda

# Runnable只能接收一個變數。所以當你要送入複數變數的時候，請用dictionary

@chain
def config_retriever(kwargs):

    # 在實際上的軟體工程這會比較複雜一些，因為有時候不見得會有條件
    
    query = kwargs['query']
    filter_ = Filter(
    must=[
        FieldCondition(key="metadata.詩體", match=MatchValue(value=kwargs["poetic_form"])),
        ]
    )

    semantic_documents = retriever_QVS.invoke(query, config={"configurable":
                                                              {"qdrant_search_kwargs":{
                                                                    "k": 5,
                                                                    "filter": filter_}}})

    return semantic_documents


@chain
def document_2_context(documents):

    context = "\n\n".join([c.page_content for c in documents])

    return context
    

In [ ]:
# 一路Combo開到底

context_pipeline = config_retriever|document_2_context

poem_pipeline = RunnablePassthrough.assign(context=context_pipeline)|chat_prompt_template|model_poem|output_parser

In [ ]:
poem_pipeline.invoke({"query": query,
                      "poetic_form": "五言絕句"})

### 平行運算 (Parallelize steps)

將複數但相互獨立的流程同時啟動

In [ ]:
from langchain_core.runnables import RunnableParallel

poem_pipeline = RunnablePassthrough.assign(context=context_pipeline)|chat_prompt_template|model_poem|output_parser

poem_pipeline_1 = RunnablePassthrough.assign(poetic_form=itemgetter("poetic_form_1"))|poem_pipeline
poem_pipeline_2 = RunnablePassthrough.assign(poetic_form=itemgetter("poetic_form_2"))|poem_pipeline

map_chain = RunnableParallel(poem_1=poem_pipeline_1, poem_2=poem_pipeline_2)

map_chain.invoke({"poetic_form_1": "五言絕句", "poetic_form_2": "七言絕句", "query": query})

### 異步流 (Asynchronization)

當一個流程進入等待狀態（例如等待外部 API 回應、讀寫檔案、資料庫查詢），這個工作會「讓出執行權」給事件迴圈 (event loop)。
此時事件迴圈可以去執行其他等待中的工作，而不是卡在原地。

價值：
在高併發的 AI 應用場景（例如大量查詢、同時呼叫模型 API），異步流能顯著提升資源利用率，減少等待時間 → 提升整體吞吐量。
（注意：異步不會讓單一請求變快，而是讓系統能同時處理更多請求。）

建議的開發方法：

1. 先建立同步流程：確保邏輯正確，方便除錯。

2. 再轉成異步流程：將 I/O 密集的部分（API 呼叫、DB 存取、檔案處理）改寫成 async / await，或用 asyncio.to_thread 包裝同步函式。

3. 測試效能與正確性：確認異步版本在高並發情境下能穩定運作。

In [ ]:
@chain
async def config_retriever_async(kwargs):
    """Async retriever"""

    query = kwargs['query']
    filter_ = Filter(
        must=[
            FieldCondition(
                key="metadata.詩體",
                match=MatchValue(value=kwargs["poetic_form"])
            )
        ]
    )

    # retriever_QVS must support ainvoke()
    semantic_documents = await retriever_QVS.ainvoke(
        query,
        config={
            "configurable": {
                "qdrant_search_kwargs": {
                    "k": 5,
                    "filter": filter_,
                }
            }
        },
    )

    return semantic_documents


@chain
async def document_2_context_async(documents):
    """Async doc -> context"""
    context = "\n\n".join([c.page_content for c in documents])
    return context


# Pipeline definition (works for sync + async)
context_pipeline_async = config_retriever_async | document_2_context_async

poem_pipeline_async = (
    RunnablePassthrough.assign(context=context_pipeline)
    | chat_prompt_template
    | model_poem
    | output_parser
)

In [ ]:
result = await poem_pipeline_async.ainvoke({"query": query, "poetic_form": "五言絕句"})

In [ ]:
result

比較同步流(Synchronization)和異步流(Asynchronization)的時間

In [ ]:
from datetime import datetime

begin = datetime.now()

for _ in range(5):
    _ = poem_pipeline.invoke({"query": query, "poetic_form": "五言絕句"})

end = datetime.now()

print(end - begin)

In [ ]:
begin = datetime.now()

for _ in range(5):
    _ = await poem_pipeline_async.ainvoke({"query": query, "poetic_form": "五言絕句"})

end = datetime.now()

print(end - begin)

## Coding with LLM

如何使用大語言模型(LLM)產生代碼.

1. 在使用ChatGPT的時候，可以順利地讓語言模型生成代碼
2. 有些問題可以借助代碼的方式來得到正確的答案，像是一些數學上的計算問題。
3. 如何透過API實現用LLM生成代碼解決問題

In [ ]:
from langchain_core.output_parsers import StrOutputParser


system_template = (
    "You are a highly skilled Python developer. Your task is to generate Python code strictly based on the user's instructions.\n"
    "Leverage statistical and mathematical libraries such as `statsmodels`, `scipy`, and `numpy` where appropriate to solve the problem.\n"
    "Your response must contain only the Python code — no explanations, comments, or additional text.\n\n"
)

human_template = '{query}\n\nCode:'


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"]}}

chat_prompt = build_standard_chat_prompt_template(input_)

code_pipeline = chat_prompt|model|StrOutputParser()

In [ ]:
code = code_pipeline.invoke({"query": "Calculate the area of a circle with radius 3.8976"})

In [ ]:
print(code)

In [ ]:
import re

match = re.findall(r"python\n(.*?)\n```", code, re.DOTALL)
python_code = match[0]

exec(python_code)

代碼能夠順利執行，但如何提取出答案?

其他人應該也有過類似的問題而應該已經有人提出解決辦法了，所以我就去問ChatGPT。

### ✅ `exec(...)`

- `exec()` runs the code you give it **as if it was a Python script**.
- It does **not return values**; it just executes statements (like `import`, assignments, function definitions, etc.).

---

### ✅ `{}` — the *global namespace*

- An **empty dictionary** is passed as the *global namespace*.
- This isolates the execution from your actual global scope, which is **good for sandboxing** and avoiding side effects.

---

### ✅ `local_vars` — the *local namespace*

- This dictionary collects all **local variables** defined during execution.
- After the code runs, `local_vars` will contain all the variables and their values.

```python
{
  'np': <module 'numpy'...>,
  'radius': 3.8976,
  'area': 47.73155744152567
}

In [ ]:
python_code

In [ ]:
lines = python_code.strip().split('\n')
*stmts, last_line = lines

In [ ]:
lines

In [ ]:
local_vars = {}
exec('\n'.join(stmts), {}, local_vars)

In [ ]:
local_vars

In [ ]:
import math

math.pi * 3.8976 * 3.8976

What if we calculate the area without the code part?

In [ ]:
model.invoke("Calculate the area of a circile with radius 3.8976")

### Combine the code generation and code execution in LCEL

生成代碼 -> 執行代碼

In [ ]:
from langchain_core.runnables import chain


@chain
def code_execution(code):

    match = re.findall(r"python\n(.*?)\n```", code, re.DOTALL)
    python_code = match[0]
    
    lines = python_code.strip().split('\n')
    *stmts, last_line = lines

    local_vars = {}
    exec('\n'.join(stmts), {}, local_vars)

    return local_vars


code_pipeline = chat_prompt|model|StrOutputParser()|code_execution

In [ ]:
17 * 89

In [ ]:
17 * 7

In [ ]:
code_pipeline.invoke("What is the GCD of 1513 and 119?")

In [ ]:
import numpy

5 * 10 * np.sin(35/180 * math.pi) / 2

In [ ]:
answer = code_pipeline.invoke("What is the area of a triangle, with two sides with length 10 and 5, and the angle between them is 35 degrees?")

In [ ]:
answer

能夠同時回傳問題，答案，過程嗎?

In [ ]:
code_pipeline = RunnablePassthrough.assign(code=chat_prompt|model|StrOutputParser())|RunnablePassthrough.assign(answer=itemgetter('code')|code_execution)

國小經典題目: 雞兔同籠。講真的我國小時完全不知道這怎麼做。

In [ ]:
answer = code_pipeline.invoke({"query":"現有一籠子，裡面有雞和兔子若干隻，數一數，共有頭14個，腿38條。請問一共有幾隻雞?"})

In [ ]:
answer

In [ ]:
answer = code_pipeline.invoke({"query":"在一副52張牌的常規樸克牌中，任意選出五張，得到full house的機率是多少?請用分數(numerator/denominator)的方式呈現答案。"})

In [ ]:
answer

有辦法做抽象代數嗎?
1. 小角度單擺: 給繩長L， 重力常數g。求單擺週期
2. 給予地球質量M，萬有引力係數G，地球半徑R。求從地面脫離地球重力場影響的最低初始速度。

In [ ]:
answer = code_pipeline.invoke("小角度單擺: 給繩長L， 重力常數g。求單擺週期。Give me the analytical expression.")
print(answer)

In [ ]:
answer = code_pipeline.invoke("給予地球質量M，萬有引力係數G，地球半徑R。求從地面脫離地球重力場影響的最低初始速度。Give me the analytical expression.")
print(answer)

這種有標準答案的題目其實非常適合測試LLM，因為只有對錯，沒有立場。
像是剛剛用LLM生成詩，在格式的方面可以檢驗(平仄規則啥的)，但是內容品質很難檢驗。
至於複雜的文章，可能品質檢驗就更困難了。

那剛剛的生成結果有另一個問題，那就是答案的key都不固定。這會導致無法規模化生成，那我們要如何控制輸出來進行規模化生成?
想像你是一個高中數學老師，段考題大概一次是40題，你總不希望一直要copy/paste...

In [ ]:
from langchain_core.output_parsers import StrOutputParser


system_template = dedent("""You are a highly skilled Python developer. Your task is to generate Python code strictly based on the user's instructions.
                            Leverage statistical and mathematical libraries such as `statsmodels`, `scipy`, and `numpy` where appropriate to solve the problem.
                            Your response must contain only the Python code — no explanations, comments, or additional text.
                         """
)

# 多加一行 'Always copy the final answer to a variable `answer`
# 沒有訣竅，多試就行
human_template = dedent("""{query}\n\n
                            Always copy the final answer to a variable `answer`
                            Code:
                        """)


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"]}}

chat_prompt = build_standard_chat_prompt_template(input_)

code_pipeline = chat_prompt|model|StrOutputParser()

In [ ]:
code = code_pipeline.invoke("What is the GCD of 1513 and 119?")

In [ ]:
code

In [ ]:
# 你看到產生的結果改變了，所以函數code_execution也要跟著改變

@chain
def code_execution(code):
    
    match = re.findall(r"python\n(.*?)\n```", code, re.DOTALL)
    python_code = match[0]
    
    lines = python_code.strip()#.split('\n')
    # *stmts, last_line = lines

    local_vars = {}
    exec(lines, {}, local_vars)

    return local_vars

code_execution.invoke(code)

In [ ]:
code = code_pipeline.invoke("小角度單擺: 給繩長L， 重力常數g。求單擺週期。Give me the analytical expression.")

In [ ]:
code_execution.invoke(code)

In [ ]:
code_execution.invoke(code)['answer']

# Model Evaluation

> 🎯 **本章學完你將能學會什麼：**
> - 理解「模型評估（Model Evaluation）」在 AI 專案生命週期中的關鍵角色  
> - 能分辨並監測三種常見的模型變化類型：  
>   - **Model Drift**：模型行為隨版本或訓練資料改變的偏移  
>   - **Data Drift**：輸入數據分佈隨時間演化而產生的變異  
>   - **Model Resilience**：模型面對干擾（perturbation）時的穩定性與魯棒性    
> - 能實作 **Input Perturbation（輸入干擾）** 技術，模擬實際噪音情境  
>   - 文字錯誤（typos, case change, unicode homoglyphs）  
>   - 同義改寫（paraphrasing）  
>   - 隨機噪音與對抗攻擊（adversarial perturbations）  
> - 理解 **Consistency Checking** 的概念：如何不依賴人工評估進行自我驗證  
>   - 輸出語義相似度檢查（cosine similarity via embeddings）  
>   - 自洽性（Self-consistency）與多次重試一致性量測  
>   - 翻譯回譯（Round-trip Validation）策略  
> - 能運用向量化工具（如 **SBERT / HuggingFace Embeddings**）進行輸出相似度比較  
> - 學會使用 **Perplexity** 衡量文本流暢度與自然度（以 GPT-2 為基準模型）  
> - 熟悉 **同步與異步批次運算（batch / abatch）** 技巧，提高測試效率  
> - 能撰寫自動化流程：  
>   - 清潔輸入 → 污染輸入 → 產生輸出 → 嵌入相似度 → Perplexity 評估  
> - 具備分析模型在擾動下語義穩定性與語言流暢性的能力  
> - 能以資料科學思維建立自動化模型評估系統，為模型持續優化提供量化依據  



最近在找工作。評估(Evaluation)算是一個蠻被看重的技能。

  1. Model Drift (模型隨著迭代的變化)
     - 建立Golden Benchmark -> 測量輸出結果隨模型改變的變化
  2. Data Drift (數據隨著時間的變化)
     - 量測數據的變化
  3. Model Resiliece (模型有多Robust)
     - 建立Golen Benchmark -> 干擾輸入數據，量測輸出和原始輸出的差異

## Model Resilience

In [ ]:
from IPython.display import display, HTML

# Define the HTML to display images side by side
html = """
<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="Model_resilience.png" height="600" width="1200" />
    </div>
</div>
"""

# Display the HTML
display(HTML(html))

### Input Perturbation (Automatic Input Variation) 

- Text corruption: Introduce typos, spelling variations, case changes, unicode homoglyphs.

- Paraphrasing: Use another LLM or rule-based synonym replacement to rephrase prompts.

- Noise injection: Add irrelevant but related sentences or random tokens.

- Adversarial perturbations: Apply known NLP attack techniques (e.g., TextFooler, DeepWordBug).

### Consistency Checking (Self-Evaluation without Humans)

- Semantic similarity of outputs: Use embeddings (e.g., cosine similarity via SBERT, OpenAI embeddings) to compare outputs across perturbed inputs. Robust pipelines should produce semantically close answers.

- Round-trip validation: For tasks like translation, translate back and compare with original.

- Self-consistency: Run the pipeline multiple times with the same input (temperature > 0) and measure variance in answers.

#### Example: Text Corruption

In [ ]:
"""
出處: Warhammer 40k: The Horus Heresy
"""

cleaned_inputs = dedent("""
                  I never wanted this, I never wanted to unleash my legions.
                  
                  Together we banished the ignorance of Old Night, but you betrayed me, you betrayed us all.

                  You stole power from the Gods and lied to your sons.

                  Mankind has only one chance to prosper, if you will not seize it then I will.

                  So let it be war, from the skies of Terra to the Galactic rim.

                  Let the seas boil, let the stars fall.

                  Though it takes the last drop of my blood, I will see the galaxy freed once more and 
                  if I can not save it from your failure Father, then let the galaxy burn!
                  """)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 調高溫度，因為我們會希望結果出現一些變化

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", temperature=0.3)

system_template = dedent("""
    You are a helpful AI assistant and you are responsible for language model resilience test.
    You will introduce typos, spelling variations, case changes, unicode homoglyphs to the given text.
""")

human_template = '{query}'


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"]}}

chat_prompt = build_standard_chat_prompt_template(input_)

corruption_pipeline = chat_prompt|model|StrOutputParser()

In [ ]:
corruption_pipeline.invoke({"query": cleaned_inputs})

In [ ]:
corruption_pipeline.invoke({"query": cleaned_inputs})

- Cleaned input -> Cleaned output
- Dirty input -> "Dirty" output

#### Cosine similarity

使用 Cosine similarity衡量輸出的相似度

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", temperature=0.0)

# 通常是由你的項目主題決定這裡要做啥

system_template = dedent("""
    You are a helpful AI assistant and you are responsible for guess the story beind the monologue.
""")

human_template = '{query}'


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"]}}

chat_prompt = build_standard_chat_prompt_template(input_)

semantic_pipeline = chat_prompt|model|StrOutputParser()

In [ ]:
# 使用一個英文的embedding，這個比較小一些

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
cleaned_output = semantic_pipeline.invoke({"query": cleaned_inputs})

In [ ]:
# 汙染來源
dirty_input = corruption_pipeline.invoke({"query": cleaned_inputs})

# 根據汙染後的數據產出結果
dirty_output = semantic_pipeline.invoke({"query": dirty_input})

In [ ]:
# 計算乾淨產出的embedding

embedding_cleaned_output = embedding.embed_documents([cleaned_output])

In [ ]:
# 計算汙染後產出的embedding

embedding_dirty_output = embedding.embed_documents([dirty_output])

In [ ]:
# 比較兩者的相似程度

cosine_similarity(embedding_cleaned_output, embedding_dirty_output)

In [ ]:
# 一個毫不相關的比較

random_embedding = embedding.embed_documents(["The weather is wonderful and we are going to hiking in the nearby forest."])

cosine_similarity(embedding_cleaned_output, random_embedding)

In [ ]:
# Generate a batch of pertubed inputs

batch = [cleaned_inputs] * 10

corrupted_data = await corruption_pipeline.abatch(batch)

In [ ]:
corrupted_data

In [ ]:
import pandas as pd

df = pd.DataFrame(data = [[a] for a in corrupted_data], columns=['dirty_input'])

In [ ]:
df

In [ ]:
df.to_dict("records")

In [ ]:
dirty_output = await semantic_pipeline.abatch(df.rename(columns={"dirty_input": "query"}).to_dict("records"))

In [ ]:
df['dirty_output'] = dirty_output

### Perplexity Measurement:

對於輸入進行干擾後，利用Perplexity (不是那個AI搜索公司) 計算輸出的流暢性

Perplexity 數值越高，表示模型對這段文字感到『越困惑』，亦即這段文字的流暢性或自然度越低；反之，數值越低則越流暢

這裡我們使用GPT2模型作為量測工具。硬體夠力的話也可以選擇更大的模型像是Llama-3。

如果 Cosine Similarity 很高，但 Perplexity 差距很大，代表模型在受到干擾後輸出的語意沒變，但語言組織能力下降了。

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def calculate_perplexity(text):
    encodings = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()

In [ ]:
print(calculate_perplexity(cleaned_output))

In [ ]:
df['perplexity'] = df['dirty_output'].apply(lambda x: calculate_perplexity(x))

In [ ]:
df

### Model Drift

固定住輸入，然後使用不同的模型輸出。最後比較輸出。